In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

players = pd.read_csv("../data/player_profiles.csv")
performances = pd.read_csv("../data/player_performances.zip", compression='zip')
market_values = pd.read_csv("../data/player_market_value.csv")
injuries = pd.read_csv("../data/player_injuries.csv")
teams = pd.read_csv("../data/team_details.csv")
transfers = pd.read_csv("../data/transfer_history.csv")

C:\Users\morga\AppData\Local\Temp\ipykernel_25148\1939850787.py:5: DtypeWarning: Columns (29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv("../data/player_profiles.csv")


In [ ]:
import pandas as pd
import numpy as np

# ============================================================================
# IDENTIFY JOIN KEY
# ============================================================================

# ⚠️ CHANGE THIS to match your actual column name
PLAYER_ID_COL = 'player_id'

# ============================================================================
# AGGREGATE PERFORMANCE DATA
# ============================================================================

# Filter to most recent season if available
if 'season' in performances.columns:
    latest_season = performances['season'].max()
    performances_recent = performances[performances['season'] == latest_season]
else:
    performances_recent = performances

# Aggregate performance stats per player
# ⚠️ ADJUST column names to match your CSV
performance_agg = performances_recent.groupby(PLAYER_ID_COL).agg({
    'matches': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'minutes_played': 'sum',
    'yellow_cards': 'sum',
    'red_cards': 'sum',
}).reset_index()

# Create derived features
performance_agg['goals_per_90'] = (performance_agg['goals'] / performance_agg['minutes_played'] * 90).round(2)
performance_agg['assists_per_90'] = (performance_agg['assists'] / performance_agg['minutes_played'] * 90).round(2)
performance_agg['goal_contributions'] = performance_agg['goals'] + performance_agg['assists']

# Handle division by zero
performance_agg['goals_per_90'].fillna(0, inplace=True)
performance_agg['assists_per_90'].fillna(0, inplace=True)

# ============================================================================
# GET LATEST MARKET VALUE
# ============================================================================

# Find date column
date_column_options = ['date', 'valuation_date', 'datetime', 'timestamp']
date_col = None

for col in date_column_options:
    if col in market_values.columns:
        date_col = col
        break

if date_col:
    market_values[date_col] = pd.to_datetime(market_values[date_col])
    
    # Get most recent valuation per player
    latest_values = market_values.sort_values(date_col).groupby(PLAYER_ID_COL).tail(1)
    
    # Calculate value change
    first_values = market_values.sort_values(date_col).groupby(PLAYER_ID_COL).head(1)
    
    value_change = latest_values.merge(
        first_values[[PLAYER_ID_COL, 'market_value']], 
        on=PLAYER_ID_COL, 
        suffixes=('_latest', '_first'),
        how='left'
    )
    
    value_change['value_change_pct'] = (
        (value_change['market_value_latest'] - value_change['market_value_first']) / 
        value_change['market_value_first'] * 100
    ).round(2)
    
    latest_values = latest_values.merge(
        value_change[[PLAYER_ID_COL, 'value_change_pct']], 
        on=PLAYER_ID_COL, 
        how='left'
    )
else:
    latest_values = market_values.copy()
    latest_values['value_change_pct'] = 0

# Rename market value column
if 'market_value' in latest_values.columns:
    latest_values = latest_values.rename(columns={'market_value': 'current_market_value'})
elif 'value' in latest_values.columns:
    latest_values = latest_values.rename(columns={'value': 'current_market_value'})

# ============================================================================
# AGGREGATE INJURY DATA
# ============================================================================

# ⚠️ ADJUST column names to match your CSV
injury_agg = injuries.groupby(PLAYER_ID_COL).agg({
    'days_injured': 'sum',
    'injury_id': 'count'
}).reset_index()

injury_agg = injury_agg.rename(columns={
    'days_injured': 'total_days_injured',
    'injury_id': 'total_injuries'
})

# Create injury proneness score
injury_agg['injury_proneness'] = (injury_agg['total_injuries'] / 3).round(2)

# ============================================================================
# JOIN ALL DATAFRAMES
# ============================================================================

# Start with player profiles
df = players.copy()

# Join performance stats
df = df.merge(performance_agg, on=PLAYER_ID_COL, how='left')

# Join market values (INNER JOIN - only keep players with valuations)
df = df.merge(
    latest_values[[PLAYER_ID_COL, 'current_market_value', 'value_change_pct']], 
    on=PLAYER_ID_COL, 
    how='inner'
)

# Join injury data
df = df.merge(injury_agg, on=PLAYER_ID_COL, how='left')

# Fill missing injury data with 0
df['total_days_injured'].fillna(0, inplace=True)
df['total_injuries'].fillna(0, inplace=True)
df['injury_proneness'].fillna(0, inplace=True)

# Join team details if team_id exists
if 'team_id' in df.columns and 'team_id' in teams.columns:
    team_cols = ['team_id']
    if 'league' in teams.columns:
        team_cols.append('league')
    if 'team_market_value' in teams.columns:
        team_cols.append('team_market_value')
    
    df = df.merge(teams[team_cols], on='team_id', how='left')

# ============================================================================
# DATA CLEANING
# ============================================================================

# Define critical columns
critical_columns = ['current_market_value']
if 'age' in df.columns:
    critical_columns.append('age')
if 'position' in df.columns:
    critical_columns.append('position')

# Remove rows with missing critical data
df = df.dropna(subset=critical_columns)

# Remove players with zero market value
df = df[df['current_market_value'] > 0]

# Fill missing performance stats with 0
performance_cols = ['goals', 'assists', 'minutes_played', 'matches', 
                   'goals_per_90', 'assists_per_90', 'goal_contributions',
                   'yellow_cards', 'red_cards']

for col in performance_cols:
    if col in df.columns:
        df[col].fillna(0, inplace=True)

# ============================================================================
# SAVE DATASET
# ============================================================================

df.to_csv("player_valuation_dataset.csv", index=False)

# Display summary
print(f"Dataset created: {len(df):,} players × {len(df.columns)} columns")
print(f"\nSample data:")
print(df.head())
print(f"\nBasic statistics:")
print(df[['current_market_value', 'goals', 'assists', 'age']].describe())